In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient()
db = client.swatford

In [3]:
db.collection_names()

['uniprot',
 'medline.descs.mappings',
 'reactome',
 'gene2pubmed',
 'mgi',
 'ctd',
 'system.indexes',
 'medline.descs.ancestors.mappings',
 'rgd',
 'generif',
 'medline.descs',
 'genes.human.mappings']

In [4]:
human_gene_map = pd.DataFrame(list(db.genes.human.mappings.find({},{"_id":0,"gene_id":1,"human_gene_id":1})))

In [13]:
uniprot = pd.DataFrame(list(db.uniprot.find({},{"_id":0,"gene_id":1,"pmid":1}))).merge(human_gene_map).drop_duplicates()

In [15]:
reactome = pd.DataFrame(list(db.reactome.find({},{"_id":0,"gene_id":1,"pmid":1}))).merge(human_gene_map).drop_duplicates()

In [17]:
gene2pubmed = pd.DataFrame(list(db.gene2pubmed.find({},{"_id":0,
                                                    "gene_id":1,"pmid":1}))).merge(human_gene_map).drop_duplicates()

In [18]:
mgi = pd.DataFrame(list(db.mgi.find({},{"_id":0,"gene_id":1,"pmid":1}))).merge(human_gene_map).drop_duplicates()

In [19]:
ctd = pd.DataFrame(list(db.ctd.find({},{"_id":0,"gene_id":1,"pmid":1}))).merge(human_gene_map).drop_duplicates()

In [20]:
rgd = pd.DataFrame(list(db.rgd.find({},{"_id":0,"gene_id":1,"pmid":1}))).merge(human_gene_map).drop_duplicates()

In [21]:
generif = pd.DataFrame(list(db.generif.find({},{"_id":0,"gene_id":1,"pmid":1}))).merge(human_gene_map).drop_duplicates()

In [22]:
gene_pmid = pd.concat([uniprot,reactome,gene2pubmed,mgi,ctd,rgd,generif]).drop_duplicates()

In [24]:
len(gene_pmid)

4786543

In [26]:
len(gene_pmid),len(gene_pmid[["gene_id","pmid"]].drop_duplicates()),len(gene_pmid[["human_gene_id","pmid"]].drop_duplicates())

(4786543, 3506689, 3797237)

In [27]:
gene_pmid = gene_pmid[["human_gene_id","pmid"]].rename(columns={"human_gene_id":"gene_id"}).drop_duplicates()

In [30]:
len(gene_pmid.pmid.unique())

894693

In [31]:
len(gene_pmid.gene_id.unique())

19282

In [32]:
db.medline.genes.mappings.insert_many(gene_pmid.to_dict(orient="records"))